In [1]:
import pandas as pd

counseling = pd.read_excel('Data/Counseling Data.xlsx')
requests = pd.read_excel('Data/Counsleing Requests.xlsx')
clients = pd.read_excel('Data/Client Data.xlsx')

apexDat = counseling[counseling['Created By'].isin(['alsaifig','cbeck', 'farleyq', 'dogav', 'bureshm', 'guinottep', 'garzonc', 'danielk', 'sargush'])]

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [2]:
requests['Start Date'] = pd.to_datetime(requests['Start Date'])
requests = requests[requests['Start Date'] <= "2024-05-24"]

requests['Month'] = requests['Start Date'].dt.strftime('%B')
requests['MonthNumber'] = requests['Start Date'].dt.month

requests['Start Date'] = requests['Start Date'].dt.date

### Counseling Requests by Month

In [3]:
import pandasql as psql

query = """
SELECT Month, COUNT(*) AS "# of Requests"
FROM requests
GROUP BY Month
ORDER BY MonthNumber
"""

monthlyRequests = psql.sqldf(query, locals())

# monthlyRequests.to_excel('Deliverables/Monthly Requests by Month.xlsx')
monthlyRequests

,Month,# of Requests
0,January,56
1,February,46
2,March,53
3,April,30
4,May,26


### Clients Wihtout Followup

In [4]:
query = """
SELECT requests.*
FROM requests
LEFT JOIN apexDat ON requests.Client = apexDat.Client
WHERE apexDat.Client IS NULL
"""

followUp = psql.sqldf(query, locals())

followUp = followUp[['Client', 'Start Date']]
followUp

,Client,Start Date
0,Pedersen Family Construction LLC (@0846791),2024-05-21
1,CUBG (@0846761),2024-05-15
2,Felicia pedrick (@0846756),2024-05-14
3,Eleden cleaning (APEX5854),2024-05-14
4,Herr Solutions (APEX5851),2024-05-13
...,...,...
65,Unique Cleaning Services Inc. (@0846136),2024-01-19
66,Conethic Consulting Services (@0846107),2024-01-10
67,Five Star Painting of Ankeny & West Des Moines...,2024-01-10
68,Wendy Dale (@0846110),2024-01-10


In [5]:
query = """
SELECT followUp."Start Date", clients.*
FROM followUp
LEFT JOIN clients ON followUp.Client = clients.Client
WHERE 
    "Physical Address State" = "Nebraska"
    AND "Primary Consultants" IN ("Patrick Guinotte", "Carlos Garzon")
"""

noFollowUp = psql.sqldf(query, locals())

noFollowUp = noFollowUp[['Client', 'Start Date', 'Primary Consultants', 'Primary Contact', 'Phone Number', 'Email Address']]

# noFollowUp.to_excel('Deliverables/Clients Requests Without Followup.xlsx')
noFollowUp

,Client,Start Date,Primary Consultants,Primary Contact,Phone Number,Email Address
0,Aubrey gutierrez (APEX5715),2024-01-31,Patrick Guinotte,Aubrey gutierrez,(402) 917-7497,aubrey.gutierrez@nebraska.gov
1,Autumn Jones (APEX5842),2024-04-29,Patrick Guinotte,Autumn Jones,(402) 999-9999,autumn.jonesm@gmail.com
2,Boston School of Cosmetology (APEX5790),2024-03-20,Patrick Guinotte,Brooke Boston,(531) 466-8444,brooke@bostonschoolofcosmetology.com
3,Eleden cleaning (APEX5854),2024-05-14,Carlos Garzon,Telma Cordova,(217) 602-6806,morenitacordova5@gmail.com
4,Erin Hagan (APEX5770),2024-03-09,Patrick Guinotte,Erin Hagan,(402) 676-3705,Erinchagan@gmail.com
5,"Glace House, LLC (APEX5804)",2024-04-03,Patrick Guinotte,Michael Vann,(402) 885-3727,glassfaucets@glacehouse.com
6,Goodwill Industries Inc Serving Southeast Nebr...,2024-03-26,Patrick Guinotte,Tyler Swain,(402) 671-4037,Development@goodwillomaha.org
7,Herr Solutions (APEX5851),2024-05-13,Patrick Guinotte,Michelle Herr,(419) 789-0293,Michelle-herr@hotmail.com
8,ideologix insights (APEX5709),2024-01-25,Patrick Guinotte,heather nelson,(402) 690-0957,pollthevote@gmail.com


### GIB/DIB Ready Clients by Month

In [6]:
clients['Client Start Date'] = pd.to_datetime(clients['Client Start Date'])
clients = clients[clients['Client Start Date'] <= pd.Timestamp("2024-05-31")].copy()
clients = clients[clients['Client Start Date'] >= pd.Timestamp("2024-01-01")].copy()

clients['GIB Ready Date'] = pd.to_datetime(clients['GIB Ready Date'])
clients['DIB Ready Date'] = pd.to_datetime(clients['DIB Ready Date'])

clients.loc[:, 'GIB Ready Date'] = clients['GIB Ready Date'].dt.date
clients.loc[:, 'DIB Ready Date'] = clients['DIB Ready Date'].dt.date

In [7]:
gib = clients[clients['Government Industrial Base (GIB) ready?'] == True].copy()

gib.loc[:, 'Month'] = gib['GIB Ready Date'].dt.strftime('%B')
gib.loc[:, 'MonthNumber'] = gib['GIB Ready Date'].dt.month

gib = gib[['Client', 'Primary Consultants', 
          'Government Industrial Base (GIB) ready?', 'GIB Ready Date', 
          'Defense Industrial Base (DIB) ready?', 'DIB Ready Date', 
          'Month', 'MonthNumber']]

gib = gib[gib['Month'].isin(['January', 'February', 'March', 'April', 'May'])]

# gib.to_excel('Deliverables/GIB Ready Clients (Jan - May 24).xlsx')
gib

,Client,Primary Consultants,Government Industrial Base (GIB) ready?,GIB Ready Date,Defense Industrial Base (DIB) ready?,DIB Ready Date,Month,MonthNumber
71,BIG BROWN BAT SOLUTIONS INC. (dba Road Sentine...,Quentin Farley,True,2024-02-13,False,NaT,February,2.0
86,Brian Perrett (APEX5802),Charles (Chuck) Beck,True,2024-04-12,False,NaT,April,4.0
131,"Confidante, LLC (APEX5824)",Quentin Farley,True,2024-05-31,False,NaT,May,5.0
138,County Line Helicopter Inc (APEX5712),Quentin Farley,True,2024-02-13,False,NaT,February,2.0
145,CUSTER PUBLIC POWER DISTRICT (APEX5707),Charles (Chuck) Beck,True,2024-02-01,False,NaT,February,2.0
179,Enterphase Consulting LLC (APEX5813),Patrick Guinotte,True,2024-04-16,False,NaT,April,4.0
216,GamTech Technologies (APEX5746),Patrick Guinotte,True,2024-03-13,False,NaT,March,3.0
259,I-CUBE TECHNOLOGIES LLC (APEX5829),Patrick Guinotte,True,2024-05-14,False,NaT,May,5.0
290,Jeremy Gumaer (APEX5809),Charles (Chuck) Beck,True,2024-05-06,False,NaT,May,5.0
314,Keystone Bar LLC (APEX5822),Benjamin Benton,True,2021-05-10,False,NaT,May,5.0


In [8]:
query = """
SELECT Month, COUNT(*) AS "# of GIBs"
FROM gib
WHERE Month IN ("January", "February", "March", "April", "May")
GROUP BY Month, MonthNumber
ORDER BY MonthNumber
"""

gibReady = psql.sqldf(query, locals())

# gibReady.to_excel('Deliverables/GIB Ready Clients by Month.xlsx')
gibReady

,Month,# of GIBs
0,January,1
1,February,5
2,March,2
3,April,9
4,May,6


In [9]:
dib = clients[clients['Defense Industrial Base (DIB) ready?'] == True].copy()

dib.loc[:, 'Month'] = dib['DIB Ready Date'].dt.strftime('%B')
dib.loc[:, 'MonthNumber'] = dib['DIB Ready Date'].dt.month

dib = dib[['Client', 'Primary Consultants', 
          'Government Industrial Base (GIB) ready?', 'GIB Ready Date', 
          'Defense Industrial Base (DIB) ready?', 'DIB Ready Date', 
          'Month', 'MonthNumber']]

dib = dib[dib['Month'].isin(['January', 'February', 'March', 'April', 'May'])]

# dib.to_excel('Deliverables/DIB Ready Clients (Jan - May 24).xlsx')
dib

,Client,Primary Consultants,Government Industrial Base (GIB) ready?,GIB Ready Date,Defense Industrial Base (DIB) ready?,DIB Ready Date,Month,MonthNumber
565,Taz Strategic Alliance Group LLC. (APEX5847),Patrick Guinotte,True,2024-05-14,True,2024-05-14,May,5
585,Thyreos Inc (APEX5827),Quentin Farley,True,2024-04-22,True,2024-04-22,April,4


In [10]:
query = """
SELECT Month, COUNT(*) AS "# of GIBs"
FROM dib
WHERE Month IN ("January", "February", "March", "April", "May")
GROUP BY Month, MonthNumber
ORDER BY MonthNumber
"""

dibReady = psql.sqldf(query, locals())

# dibReady.to_excel('Deliverables/DIB Ready Clients by Month.xlsx')
dibReady

,Month,# of GIBs
0,April,1
1,May,1


### Clients that did not become GIB/DIB ready

In [11]:
query= """
SELECT *
FROM clients
WHERE 
    "Client Start Date" BETWEEN "2024-01-01" AND "2024-05-31"
    AND "Government Industrial Base (GIB) ready?" = False
    OR "Defense Industrial Base (DIB) ready?" = False
"""

notReady = psql.sqldf(query, locals())

notReady = notReady[['Client', 'Client Start Date', 'Primary Consultants',
                'Government Industrial Base (GIB) ready?', 'GIB Ready Date',
                'Defense Industrial Base (DIB) ready?', 'DIB Ready Date']]

# notReady.to_excel('Deliverables/Clients that are not GIB or DIB Ready.xlsx')
notReady

,Client,Client Start Date,Primary Consultants,Government Industrial Base (GIB) ready?,GIB Ready Date,Defense Industrial Base (DIB) ready?,DIB Ready Date
0,KW & SONS TREE SERVICE LLC (APEX5820),2024-04-18 00:00:00.000000,Charles (Chuck) Beck,0,None,0,None
1,*Unknown at this time* (L4383),2024-05-07 00:00:00.000000,Craig Boesch,0,None,0,None
2,°Fresh Air HVAC (APEX5690),2024-01-02 00:00:00.000000,Harold Sargus,0,None,0,None
3,1404 Collective (K3506),2024-05-14 00:00:00.000000,Odee Ingersoll,0,None,0,None
4,1st Serve Pickleball (O11507),2024-01-08 00:00:00.000000,Ola Babalola,0,None,0,None
...,...,...,...,...,...,...,...
612,"Young's General Contracting, Inc. (@0846648)",2024-05-06 00:00:00.000000,Veronica Doga,0,None,0,None
613,YouStudioTv Inc (SBIR0770),2024-05-14 00:00:00.000000,Josh Nichol-Caddy,0,None,0,None
614,YUYI FENCE AND DECKS LLC (@0846764),2024-05-15 00:00:00.000000,None,0,None,0,None
615,ZAMAM PROPERTIES LLC (APEX5732),2024-02-12 00:00:00.000000,Meghann Buresh,0,None,0,None
